In [1]:
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential, Model

import tensorflow as tf
tf.compat.v1.disable_eager_execution()
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Lambda, Input, Dense
from tensorflow.keras.losses import mse, binary_crossentropy, kl_divergence
from tensorflow.keras import optimizers
from tensorflow.keras import backend as K

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, PowerTransformer

import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import random
import numpy as np
from sklearn.manifold import TSNE
# processing imports
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split,StratifiedKFold,GridSearchCV
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

from joblib import dump, load
from sklearn.metrics import accuracy_score, f1_score, precision_score,recall_score
from sklearn.linear_model import Perceptron
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC


from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp
from itertools import cycle
import seaborn as sns
from sklearn.datasets import make_classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
import sklearn.metrics as metrics

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# Any results you write to the current directory are saved as output.
%matplotlib inline

In [2]:
columns = (['duration'
,'protocol_type'
,'service'
,'flag'
,'src_bytes'
,'dst_bytes'
,'land'
,'wrong_fragment'
,'urgent'
,'hot'
,'num_failed_logins'
,'logged_in'
,'num_compromised'
,'root_shell'
,'su_attempted'
,'num_root'
,'num_file_creations'
,'num_shells'
,'num_access_files'
,'num_outbound_cmds'
,'is_host_login'
,'is_guest_login'
,'count'
,'srv_count'
,'serror_rate'
,'srv_serror_rate'
,'rerror_rate'
,'srv_rerror_rate'
,'same_srv_rate'
,'diff_srv_rate'
,'srv_diff_host_rate'
,'dst_host_count'
,'dst_host_srv_count'
,'dst_host_same_srv_rate'
,'dst_host_diff_srv_rate'
,'dst_host_same_src_port_rate'
,'dst_host_srv_diff_host_rate'
,'dst_host_serror_rate'
,'dst_host_srv_serror_rate'
,'dst_host_rerror_rate'
,'dst_host_srv_rerror_rate'
,'attack'
,'difficulty_level'])

In [3]:
def subsample(df):
    df_malicious = df[df['binary_attack'] == 1]
    df_benign= df[df['binary_attack'] == 0]
    
    print('Before subsampling: ', df_benign.shape, df_malicious.shape)
    
    df_benign = df_benign.sample(len(df_malicious), random_state = 0)
    df = pd.concat([df_malicious, df_benign])
    df = df.sample(frac=1, random_state=0)
    
    print('After subsampling: ', df_benign.shape, df_malicious.shape)
    
    return df

In [4]:
def tsne_plot(x1, y1):
    tsne = TSNE(n_components=2, random_state=0)
    X_t = tsne.fit_transform(x1)

    plt.figure(figsize=(12, 8))
    plt.scatter(X_t[np.where(y1 == 0), 0], X_t[np.where(y1 == 0), 1], marker='o', color='g', linewidth='1', alpha=0.8, label=0)
    plt.scatter(X_t[np.where(y1 == 1), 0], X_t[np.where(y1 == 1), 1], marker='o', color='r', linewidth='1', alpha=0.8, label=1)

    plt.legend(loc='best');
    #plt.savefig(name);
    plt.show();

In [5]:
def report_heatmap(y_pred):
    
    # overall accuracy 
    acc = accuracy_score(y_test, y_pred)
    print( f'Overall Accuracy: {acc*100:.2f}%' )
    print("--------------------------")    
    
    # report
    print(classification_report(y, y_pred))
    figsize=(8,5)
    #y_pred = y_pred.astype(int)
    cm = confusion_matrix(y_test, y_pred, labels=np.unique(y_test))
    cm_sum = np.sum(cm, axis=1, keepdims=True)
    cm_perc = cm / cm_sum.astype(float) * 100
    annot = np.empty_like(cm).astype(str)
    nrows, ncols = cm.shape
    for i in range(nrows):
        for j in range(ncols):
            c = cm[i, j]
            p = cm_perc[i, j]
            if i == j:
                s = cm_sum[i]
                annot[i, j] = '%.1f%%\n%d/%d' % (p, c, s)
            elif c == 0:
                annot[i, j] = ''
            else:
                annot[i, j] = '%.1f%%\n%d' % (p, c)
    cm = pd.DataFrame(cm, index=np.unique(y_test), columns=np.unique(y_test))
    cm.index.name = 'Actual'
    cm.columns.name = 'Predicted'
    fig, ax = plt.subplots(figsize=figsize)
    
    sns.heatmap(cm, cmap= "YlGnBu", annot=annot, fmt='', ax=ax)

In [6]:
df_train=pd.read_csv('kdd_train.txt',header=None,names=columns)

In [7]:
df_train.shape

(125973, 43)

In [8]:
#accessing names of training columns
lst_names = df_train.columns
lst_names


Index(['duration', 'protocol_type', 'service', 'flag', 'src_bytes',
       'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot',
       'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell',
       'su_attempted', 'num_root', 'num_file_creations', 'num_shells',
       'num_access_files', 'num_outbound_cmds', 'is_host_login',
       'is_guest_login', 'count', 'srv_count', 'serror_rate',
       'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate',
       'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count',
       'dst_host_srv_count', 'dst_host_same_srv_rate',
       'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
       'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
       'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
       'dst_host_srv_rerror_rate', 'attack', 'difficulty_level'],
      dtype='object')

In [9]:
# We will make the attack column binomial by classifying all attacks as one i.e. normal flow vs attack
df_train["binary_attack"]=df_train.attack.map(lambda a: 0 if a == 'normal' else 1)
df_train.drop('attack',axis=1,inplace=True)

In [10]:
df_train['binary_attack'].value_counts()

0    67343
1    58630
Name: binary_attack, dtype: int64

In [11]:
#One-hot encoding
def one_hot(df, cols):
    """
    @param df pandas DataFrame
    @param cols a list of columns to encode
    @return a DataFrame with one-hot encoding
    """
    for each in cols:
        dummies = pd.get_dummies(df[each], prefix=each, drop_first=False)
        df = pd.concat([df, dummies], axis=1)
        df = df.drop(each, 1)
    return df

In [12]:
#defining col list
cols = ['protocol_type','service','flag']
cols

['protocol_type', 'service', 'flag']

In [13]:
df=one_hot(df_train,cols)

C:\Users\12403\AppData\Local\Temp\ipykernel_8988\445847675.py:11: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop(each, 1)
C:\Users\12403\AppData\Local\Temp\ipykernel_8988\445847675.py:11: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop(each, 1)
C:\Users\12403\AppData\Local\Temp\ipykernel_8988\445847675.py:11: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop(each, 1)


In [14]:
df.shape

(125973, 124)

In [15]:
df.drop('difficulty_level',axis = 1,inplace = True)
#Train Data

df.drop('num_outbound_cmds',axis = 1,inplace = True)
df.drop('num_root',axis = 1,inplace = True)

#This variable is highly correlated with serror_rate and should be ignored for analysis.
#(Correlation = 0.9983615072725952)
df.drop('srv_serror_rate',axis = 1,inplace = True)

#This variable is highly correlated with rerror_rate and should be ignored for analysis.
#(Corredfation = 0.9947309539817937)
df.drop('srv_rerror_rate',axis = 1, inplace=True)

#This variable is highly correlated with srv_serror_rate and should be ignored for analysis.
#(Correlation = 0.9993041091850098)
df.drop('dst_host_srv_serror_rate',axis = 1, inplace=True)

#This variable is highly correlated with rerror_rate and should be ignored for analysis.
#(Correlation = 0.9869947924956001)
df.drop('dst_host_serror_rate',axis = 1, inplace=True)

#This variable is highly correlated with srv_rerror_rate and should be ignored for analysis.
#(Correlation = 0.9821663427308375)
df.drop('dst_host_rerror_rate',axis = 1, inplace=True)

In [16]:
df.shape

(125973, 116)

In [17]:
original_df=subsample(df)

Before subsampling:  (67343, 116) (58630, 116)
After subsampling:  (58630, 116) (58630, 116)


In [18]:
normal = original_df[original_df['binary_attack'] == 0].sample(3000)
anomaly = original_df[original_df['binary_attack'] == 1].sample(3000)

df = normal.append(anomaly).sample(frac=1).reset_index(drop=True)


C:\Users\12403\AppData\Local\Temp\ipykernel_8988\435235637.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = normal.append(anomaly).sample(frac=1).reset_index(drop=True)


In [19]:
X = df.drop(['binary_attack'], axis = 1).values
Y = df["binary_attack"].values

In [20]:
X.shape

(6000, 115)

In [21]:
from tensorflow.python.keras import regularizers
## input layer 
input_layer = Input(shape=(X.shape[1],))

## encoding part
encoded = Dense(100, activation='tanh', activity_regularizer=regularizers.l1(10e-5))(input_layer)
encoded = Dense(50, activation='relu')(encoded)

## decoding part
decoded = Dense(50, activation='tanh')(encoded)
decoded = Dense(100, activation='tanh')(decoded)

## output layer
output_layer = Dense(X.shape[1], activation='relu')(decoded)

In [22]:
X.shape[1]

115

In [23]:
autoencoder = Model(input_layer, output_layer)
autoencoder.compile(optimizer="adam", loss="mse")

In [24]:
from sklearn import preprocessing
x = original_df.drop(["binary_attack"], axis=1)
y = original_df["binary_attack"].values

x_scale = preprocessing.MinMaxScaler().fit_transform(x.values)
x_norm, x_anomaly = x_scale[y == 0], x_scale[y == 1]

In [25]:
x_scale.shape

(117260, 115)

In [26]:
x_norm.shape

(58630, 115)

In [27]:
autoencoder.fit(x_norm[0:5000], x_norm[0:5000], 
                batch_size = 256, epochs = 10, 
                shuffle = True, validation_split = 0.20);

Train on 4000 samples, validate on 1000 samples
Epoch 1/10
4000/4000 [==============================] - 1s 237us/sample - loss: 0.0365 - val_loss: 0.0187
Epoch 2/10
4000/4000 [==============================] - 0s 26us/sample - loss: 0.0145 - val_loss: 0.0114


C:\Users\12403\AppData\Roaming\Python\Python39\site-packages\keras\engine\training_v1.py:2332: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


Epoch 3/10
4000/4000 [==============================] - 0s 25us/sample - loss: 0.0090 - val_loss: 0.0075
Epoch 4/10
4000/4000 [==============================] - 0s 25us/sample - loss: 0.0065 - val_loss: 0.0061
Epoch 5/10
4000/4000 [==============================] - 0s 25us/sample - loss: 0.0055 - val_loss: 0.0053
Epoch 6/10
4000/4000 [==============================] - 0s 25us/sample - loss: 0.0048 - val_loss: 0.0046
Epoch 7/10
4000/4000 [==============================] - 0s 24us/sample - loss: 0.0041 - val_loss: 0.0040
Epoch 8/10
4000/4000 [==============================] - 0s 24us/sample - loss: 0.0037 - val_loss: 0.0037
Epoch 9/10
4000/4000 [==============================] - 0s 25us/sample - loss: 0.0034 - val_loss: 0.0034
Epoch 10/10
4000/4000 [==============================] - 0s 24us/sample - loss: 0.0032 - val_loss: 0.0032


I am going to obtain the latent repreentation of the autoencoders for the dimentionality reduction.

It will reduce the dimentionality from 115 to 50

In [28]:
latent_representation = Sequential()
latent_representation.add(autoencoder.layers[0])
latent_representation.add(autoencoder.layers[1])
latent_representation.add(autoencoder.layers[2])

In [29]:
norm_hid_rep = latent_representation.predict(x_norm[:10000])
anomaly_hid_rep = latent_representation.predict(x_anomaly[:10000])

C:\Users\12403\AppData\Roaming\Python\Python39\site-packages\keras\engine\training_v1.py:2356: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


In [30]:
norm_hid_rep.shape

(10000, 50)

In [32]:
rep_x = np.append(norm_hid_rep, anomaly_hid_rep, axis = 0)
y_n = np.zeros(norm_hid_rep.shape[0])
y_a = np.ones(anomaly_hid_rep.shape[0])
rep_y = np.append(y_n, y_a)

In [33]:
rep_x.shape

(20000, 50)

In [34]:
rep_y.shape

(20000,)

In [35]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
train_x, val_x, train_y, val_y = train_test_split(rep_x, rep_y, test_size=0.25)
clf = LogisticRegression(solver="lbfgs").fit(train_x, train_y)
pred_y = clf.predict(val_x)

print ("")
print ("Classification Report: ")
print (classification_report(val_y, pred_y))

print ("")
print ("Accuracy Score: ", accuracy_score(val_y, pred_y))


Classification Report: 
              precision    recall  f1-score   support

         0.0       0.97      0.98      0.97      2544
         1.0       0.98      0.97      0.97      2456

    accuracy                           0.97      5000
   macro avg       0.97      0.97      0.97      5000
weighted avg       0.97      0.97      0.97      5000


Accuracy Score:  0.972
